In [ ]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
import ConnectionConfig as cc
cc.setupEnvironment()
spark = cc.startLocalCluster("Kafka")
spark.getActiveSession()
wordcount_delta_path = "./spark-warehouse/kafkaWordCount"
#spark.sparkContext.setLogLevel("DEBUG")


## Kafka
In this cell reading from a Kafka topic is initiated.
The enabled code gets the data from a docker container running Kafka.


In [2]:
df = (
     spark.readStream
     .format("kafka")
     .option("kafka.bootstrap.servers", "127.0.0.1:9092")
     .option("subscribe", "demo")
     #.option("startingOffsets", "earliest")
     .load()
 )

### Converting the data to a dataframe
json_tuple is an sql function that converts a json string to dataframe columns. The columns are named c0, c1, c2.


In [ ]:
from pyspark.sql import functions as F

df  = df.selectExpr("json_tuple(CAST(value AS STRING), 'event_time', 'word', 'count')")
df.writeStream.format("console").outputMode("update").start("kafkaTest")

#df.writeStream.format("console").outputMode("update").option("checkpointLocation",".\checkpoints").start("kafkaTest")
# inter = df.selectExpr("json_tuple(CAST(value as STRING), 'timestamp', 'key', 'value')")
# inter.printSchema()
#stream = inter.writeStream.format("console").outputMode("update").option("checkpointLocation",".\checkpoints").start("kafkaTest")

A string is converted to a timestamp and column names are changed to eventtime, key and value.

In [ ]:
wordCountEvents= inter.selectExpr("cast(c0 as timestamp) as eventtime" , "c1 as key", "INT(c2) as value").withWatermark("eventtime",'10 seconds')
wordCountEvents.printSchema()
stream = wordCountEvents.writeStream.format("console").outputMode("update").option("checkpointLocation",".\checkpoints1").start("kafkaTest")


### Wordcount aggregation
The data is aggregated by a 10 second window and the key. The result is written to a delta table.
The results are written to a delatable in append mode. This means that the data is added to the table after the window is closed after the watermark is passed.

In [ ]:
groupedEvents =wordCountEvents.groupBy(F.window("eventtime", "10 seconds", "10 seconds"), "key").sum("value").withColumnRenamed("sum(value)", "count")
groupedEvents.printSchema()
stream = groupedEvents.writeStream.format("delta").outputMode("append").option("checkpointLocation",".\checkpoints").option("path", wordcount_delta_path).start()
#stream = groupedEvents.writeStream.format("console").outputMode("update").option("checkpointLocation",".\checkpoints2").start()


In [12]:
stream.stop()

In [10]:
spark.stop()